In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer

import json
import io
from google.colab import files

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

tensorflow.executing_eagerly()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
uploaded = files.upload()
file_name = "data.json"
io.StringIO(uploaded["data.json"].decode("utf-8"))
data = json.loads(uploaded[file_name].decode("utf-8"))

Saving data.json to data (2).json


In [ ]:
df = pd.DataFrame(data, columns = ["text"])
df

,text
0,Mental Health Awareness Month: I have schizoaf...
1,"Not to brag, but instead of laying in bed ALL ..."
2,Thank you to the woman who rang me out at Targ...
3,I went to target and picked out birthday cards...
4,PREACH
...,...
15578,Cause I’ve tried so many in my past and I thou...
15579,"""Something has to give"" = something has to cha..."
15580,same. I mean she didn’t have to pop off but ok...
15581,My best advice is to read your reply out loud ...


In [ ]:
vds = SentimentIntensityAnalyzer()

df["label"] = df["text"].apply(lambda x: (vds.polarity_scores(x)))
df["label"] = df["label"].apply(lambda x: np.array(1) if x["compound"] > 0.1 is 1 else np.array(2) if x["compound"] < -0.1  else np.array(0))
df

,text,label
0,Mental Health Awareness Month: I have schizoaf...,0
1,"Not to brag, but instead of laying in bed ALL ...",0
2,Thank you to the woman who rang me out at Targ...,0
3,I went to target and picked out birthday cards...,0
4,PREACH,0
...,...,...
15578,Cause I’ve tried so many in my past and I thou...,0
15579,"""Something has to give"" = something has to cha...",0
15580,same. I mean she didn’t have to pop off but ok...,0
15581,My best advice is to read your reply out loud ...,0


In [ ]:
y = df["label"]
x = df["text"]
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [ ]:
num_words = 50000
output_dims = 200
tokenizer = Tokenizer(oov_token = "<OOV>", num_words = num_words)
tokenizer.fit_on_texts(x_train)

train_seqs = tokenizer.texts_to_sequences(x_train)
val_seqs = tokenizer.texts_to_sequences(x_val)

input_dims = len(tokenizer.word_index) + 1

# maxlen = max([len(x) for x in train_seqs])
maxlen = 200
train_seqs = pad_sequences(train_seqs, padding = "post", maxlen = maxlen)
val_seqs = pad_sequences(val_seqs, padding = "post", maxlen = maxlen)

print("train seqs shape: ", train_seqs.shape)
print('input dimensions: ', input_dims)
print('output dimensions: ', output_dims)

train seqs shape:  (12466, 200)
input dimensions:  16767
output dimensions:  200


In [ ]:
train_seqs = tensorflow.convert_to_tensor(train_seqs)
val_seqs = tensorflow.convert_to_tensor(val_seqs)

In [ ]:
tensorflow.keras.backend.clear_session()
model = Sequential([
    layers.Embedding(input_dim = input_dims, output_dim = 20, mask_zero = True),
    layers.GRU(20, recurrent_dropout = 0.3),
    layers.Dense(10, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(12466, activation = "softmax")
])

model.compile(optimizer = optimizers.Adam(learning_rate = 0.0005), 
              loss = "sparse_categorical_crossentropy", 
              metrics = ["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 20)          335340    
                                                                 
 gru (GRU)                   (None, 20)                2520      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 12466)             137126    
                                                                 
Total params: 475,196
Trainable params: 475,196
Non-trainable params: 0
_________________________________________________________________


In [ ]:
gru_history = model.fit(train_seqs, y_train,
                        validation_data = (val_seqs, y_val),
                        batch_size = 256,
                        epochs = 50,
                        callbacks = EarlyStopping(monitor='val_loss', patience=5))

Epoch 1/50
49/49 [==============================] - 25s 411ms/step - loss: 9.3957 - accuracy: 0.5138 - val_loss: 9.3307 - val_accuracy: 0.6164
Epoch 2/50
49/49 [==============================] - 21s 420ms/step - loss: 8.9702 - accuracy: 0.4928 - val_loss: 7.9556 - val_accuracy: 0.2396
Epoch 3/50
49/49 [==============================] - 23s 463ms/step - loss: 6.2303 - accuracy: 0.4868 - val_loss: 4.1018 - val_accuracy: 0.7894
Epoch 4/50
49/49 [==============================] - 20s 399ms/step - loss: 3.1922 - accuracy: 0.6723 - val_loss: 1.4411 - val_accuracy: 0.7894
Epoch 5/50
49/49 [==============================] - 20s 400ms/step - loss: 1.8910 - accuracy: 0.7893 - val_loss: 0.8718 - val_accuracy: 0.7894
Epoch 6/50
49/49 [==============================] - 20s 400ms/step - loss: 1.3651 - accuracy: 0.7892 - val_loss: 0.7258 - val_accuracy: 0.7894
Epoch 7/50
49/49 [==============================] - 21s 426ms/step - loss: 1.1193 - accuracy: 0.7892 - val_loss: 0.6549 - val_accuracy: 0.7894

In [ ]:
train_seqs.shape, y_train.shape, val_seqs.shape, y_val.shape

(TensorShape([12466, 200]), (12466,), TensorShape([7792, 200]), (7792,))

In [ ]:
model.predict("Thank you so much I will try my best:) hope u have an amazing day")

IndexError: ignored